# Algebraic data types

User-defined types in object-oriented languages are specified through class or trait declarations. Types can also be specified from already existing user-defined types through the inheritance mechanism.  

In functional programming, the rules for declaring new types are different: no inheritance or classes, just _products_ and _sums_ of types (and _exponentiation_, as we will see when we talk about higher-order functions). Because of the terminology and the correspondence with arithmetic (see later on), these types are called **algebraic data types**. 

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 15. Case Classes and Pattern Matching

__[Scala book (online)](https://docs.scala-lang.org/overviews/scala-book/introduction.html)__.

- [Match Expressions](https://docs.scala-lang.org/overviews/scala-book/match-expressions.html)
- [Case classes](https://docs.scala-lang.org/overviews/scala-book/case-classes.html)
- [Case objects](https://docs.scala-lang.org/overviews/scala-book/case-objects.html)

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 19. Functional Programming as Algebra 

## Product types

A value of a product type `T1 * T2` is created with a value of `T1` **and** a value of `T2`. The constructor function is:
  - `create: (T1, T2) -> T1 * T2` (create is a 2-ary function). 

Given a value of a product type, we can obtain back both values with observers:
  - `fst: T1 * T2 -> T1` 
  - `snd: T1 * T2 -> T2`
  


### Products in Scala

In Scala, products are commonly called tuples. They are defined through classes that declare one public value field (conventionally named `_1`, `_2`, etc.) for each product member type.

In [0]:
// type IntAndString = Int * String



Thus, the constructor of the product, `create`, is the constructor of the class, and the observers `fst` y `snd` the member fields `_1` y `_2`.

In [0]:
// val aProduct: IntAndString = 

### Record types

Records are like products, but we can tag the member types with a given label. For instance:

In [0]:
// type Rectangle = {width: Int * height: Int}



In [0]:
// type Circle = {radius: Int}



In [0]:
// type Triangle = {width: Int}



### Scala case classes

There are several things which are desirable to work with products/records:
 - Create new product objects without having to invoke `new`
 - Equality of product objects by value, not by reference
 - Off-the-shelf hash code
 - Pattern matching (more on this later on)


Create objects without `new`, is that possible? Yes, it is!

In [ ]:
class Rectangle(
    val width: Int,
    val height: Int)

object Rectangle{
    
}

Default equality is by-reference, not by-value:

So, we must re-define the `equals` method (rules for overriding `equals` are more [complex](https://alvinalexander.com/scala/how-to-define-equals-hashcode-methods-in-scala-object-equality), though):

In [ ]:
class Rectangle(
    val width: Int,
    val height: Int){
    
    override def equals(p1: Any): Boolean = 
       ???
}

object Rectangle{
    def apply(width: Int, height: Int): Rectangle = 
        new Rectangle(width, height)
}

Now, it works as expected:

In [ ]:
// Rectangle(5, 10) == Rectangle(5, 10)

This is tedious and repetitive, i.e. boilerplate. Scala can make it for us automatically, using so-called `case classes`: 

In [ ]:
/*
class Rectangle(width: Int, height: Int)
class Circle(radius: Int)
class Triangle(width: Int)
*/

This declaration of the `Rectangle` record is essentially equivalent to what we did manually: the `case` keyword tells the Scala compiler to generate a companion object with an `apply` constructor; override the `equals` and `hashCode` methods, among other things.

In [ ]:
// Rectangle(1,1) == Rectangle(1,1)

In [0]:
// Rectangle(1,1).hashCode

### Standard products in Scala: `TupleN` classes

The standard library of Scala has already defined for us generic case classes that represent the n-ary products (up to 22). The rough definition of `Tuple2` goes like this:

In [ ]:
object Std{
 
}

And Scala offers syntactic sugar, both for Tuple types and values. So, instead of writing something like this: 

we can write it as follows:

### Why are products called _algebraic_?

This is an example to illustrate the analogy between algebraic data types and arithmetic.

In [ ]:
// Number of values of Boolean type: 2

// Number of values of (Boolean, Boolean) type: 2 x 2 


In general, types may be regarded as sets of values. Then, the cardinal of `A * B`, for types `A` and `B` is: `|A * B| = |A| * |B|`

If product types are analogous to number multiplication, then, is there any type which corresponds to the number 1, i.e. the neutral element of the multiplication? It has to be a type `1` such that `A*1 ~= A ~= 1*A`, where the sign `~=` represents the _isomorphism_ of types, i.e. the types do not need to be equal but there should be a 1-1 mapping between the values of `A*1` and `A`.  


Since the type `1` has to comply with the identity rules, we have that `|A*1| ~= |A|`, but then `|A| * |1| = |A|`. So, `|1|` must be the type with just one value, i.e. the equivalent to the singleton set. In Scala, this type is called `Unit`.

In [ ]:
// The only value of Unit is `()`


The isomorphism between `Boolean * 1` and `Boolean` is witnessed by the following functions: 

In [ ]:
// (Boolean, Unit) ~= Boolean


which satisfy:
- `from(to(b))=b`, for all `b: Boolean`, and 
- `to(from(b))=b` for all `b: (Boolean, Unit)`.

Note that any function that returns a value of type `Unit` is completely useless from a purely functional perspective, since we already know in advance which is the (only possible) value that it returns: `()`. Therefore, if such a function makes sense is because it does something else than returning values: it must have some side effect, i.e. it has to be _impure_.

### Working with products

The goal is to translate algebraic rectangles, circles, etc., into SVG and be able to display these shapes in the notebook. We want to implement a function with the following signature:

In [ ]:
def displayCircle(c: Circle): Unit = ??? 

Note the return type: we are interested in the side effect of displaying something in the screen, so we disregard the returning value. 

We may go with the following implementation:

In [ ]:
import almond.interpreter.api.DisplayData
    
def displayCircle(c: Circle): Unit = 
   display(DisplayData(Map("text/html" -> 
        s"""<svg width="200" height="200">
              <circle cx="100" cy="100" r=${c.radius} fill="red"/>
            </svg>""")))

In [ ]:
displayCircle(Circle(100))

This works, but which are the problems of this implementation? Surely, this is the function we need, but it's not *modular* since there are at least two aspects that are intermingled in the code: 
- The computation of the SVG code (the pure part)
- The actual displaying of the HTML thunk (the impure part)

Lack of modularity in this case leads to the following problems:
- Later on, we will need to compute the SVG code of larger pictures, which are made from circles, rectangles, etc., and we will certainly need for that to translate circles to SVG. But with the current implementation, we won't be able to *reuse* anything (but by copy-pasting). 
- Another problem with this implementation is *testability*: we can't unit test the single part that generates the SVG code alone. We can only do _integration_ testing, i.e. checking the effect on the screen to see if what we get is right or not. 

So, we better modularise our function as follows:

In [ ]:
type SVG = String
type HTML = String

// Module 1: toSVG

def circleToSVG(c: Circle): SVG = 
        s"""<svg width="200" height="200">
              <circle cx="100" cy="100" r=${c.radius} fill="red"/>
            </svg>"""


// Module 2: HtmlData
import almond.interpreter.api.DisplayData

def HtmlData(svg: SVG): DisplayData = 
    DisplayData(Map("text/html" -> svg))
    

// Our very same original function, but now in a better shape
def displayCircle(c: Circle): Unit = 
    display(HtmlData(circleToSVG(c)))
    


In [ ]:
displayCircle(Circle(49))